# Feature Engineering

In [1]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
pd.options.display.float_format = '{:.2f}'.format
rc={'savefig.dpi': 75, 'figure.autolayout': False, 'figure.figsize': [12, 8], 'axes.labelsize': 18,\
   'axes.titlesize': 18, 'font.size': 18, 'lines.linewidth': 2.0, 'lines.markersize': 8, 'legend.fontsize': 16,\
   'xtick.labelsize': 16, 'ytick.labelsize': 16}

sns.set(style='dark',rc=rc)


pd.options.display.max_rows = 400
pd.options.display.max_seq_items = 200

default_color = '#56B4E9'
colormap = plt.cm.cool

In [2]:
df = pd.read_csv('data/scouts_p.csv')

In [3]:
df.columns

Index(['Assists', 'AthleteID', 'Athlete_Name', 'Away', 'ClubID',
       'Difficult_Saves', 'Fouls_Conceded', 'Fouls_Received',
       'Game_Without_Being_Scored', 'Goals', 'Goals_Conceded', 'Missed_Passes',
       'Missed_Penalty', 'Offsides', 'Opponent_Team', 'Own_Goals',
       'Participated', 'Penalty_Saves', 'Points', 'Points_avg', 'PositionID',
       'Price', 'Price_Variation', 'Red_Card', 'Round', 'Season',
       'Shot_Missed', 'Shot_Saved', 'Shot_on_Post', 'Stolen_Balls', 'Team',
       'Yellow_Card', 'opponentID'],
      dtype='object')

In [4]:
targets = ['Assists', 'Difficult_Saves', 'Fouls_Conceded', 'Fouls_Received',
       'Game_Without_Being_Scored', 'Goals', 'Goals_Conceded', 'Missed_Passes',
       'Missed_Penalty', 'Offsides', 'Own_Goals', 'Penalty_Saves', 'Points',
       'Price', 'Red_Card', 'Shot_Missed', 'Shot_Saved', 'Shot_on_Post', 
        'Stolen_Balls', 'Yellow_Card']

In [5]:
df_2014 = df[df['Season'] == 2014]
df_2015 = df[df['Season'] == 2015]
df_2016 = df[df['Season'] == 2016]
df_2017 = df[df['Season'] == 2017]

In [6]:
import time

In [7]:
'''
Function that gets the average for past available rounds for each Game Feature.
Maximum of 3 available rounds, minimum of 1.
'''
def get_avg(row, target, df):
    score = []
    athlete_id = row['AthleteID']
    match_round = row['Round']
    if(match_round == 2):
        res = df.loc[(df.AthleteID == athlete_id) & (df.Round == match_round - 1), target].values
        if(len(res) > 0):
            score.append(res[0])
        return np.mean(score)  
    if(match_round == 3):
        for i in [1,2]:
            res = df.loc[(df.AthleteID == athlete_id) & (df.Round == match_round - i), target].values
            if(len(res) > 0):
                score.append(res[0])
        return np.mean(score)   
    if(match_round > 3):
        for i in [1,2,3]:
            res = df.loc[(df.AthleteID == athlete_id) & (df.Round == match_round - i), target].values
            if(len(res) > 0):
                score.append(res[0])
        return np.mean(score)

In [8]:
# Get Average for past available rounds for each Game Feature
def get_avg_3(target, df, clubs, rounds):
    for club in clubs:
        means = df[df['ClubID'] == club].groupby(['Round','ClubID', 'PositionID'], 
                                           as_index=False).mean()
        for r in rounds:
            score = []
            if(r == 2):
                res = means.loc[(means.Round == r - 1), target].values
                if(len(res) > 0):
                    score.append(res[0])
            if(r == 3):
                for i in [1,2]:
                    res = means.loc[(means.Round == r - i), target].values
                    if(len(res) > 0):
                        score.append(res[0])
            if(r > 3):
                for i in [1,2,3]:
                    res = means.loc[(means.Round == r - i), target].values
                    if(len(res) > 0):
                        score.append(res[0])
            avg = np.mean(score)
            df.loc[((df['ClubID'] == club) & (df['Round'] == r)), 
                   'avg_{}'.format(target)] = avg

Some of the Scouts had negative values, which is an invalid score, given the number of occurrences was small, these entries are removed.

In [9]:
df_2014.min()

Assists                             0.00
AthleteID                          36443
Athlete_Name                  Abel Braga
Away                                   0
ClubID                            262.00
Difficult_Saves                     0.00
Fouls_Conceded                      0.00
Fouls_Received                      0.00
Game_Without_Being_Scored          -1.00
Goals                               0.00
Goals_Conceded                      0.00
Missed_Passes                       0.00
Missed_Penalty                      0.00
Offsides                            0.00
Opponent_Team                Atlético-MG
Own_Goals                           0.00
Participated                        True
Penalty_Saves                       0.00
Points                            -12.00
Points_avg                        -12.00
PositionID                             1
Price                               0.76
Price_Variation                    -7.24
Red_Card                            0.00
Round           

In [10]:
# Removing entries with incorrect negative values
df_2014 = df_2014[df_2014['Yellow_Card'] >= 0]
df_2014 = df_2014[df_2014['Game_Without_Being_Scored'] >= 0]

In [14]:
df_2017.min()

Assists                            -2.00
AthleteID                          36540
Athlete_Name                  Abel Braga
Away                                   0
ClubID                            262.00
Difficult_Saves                     0.00
Fouls_Conceded                     -8.00
Fouls_Received                     -5.00
Game_Without_Being_Scored           0.00
Goals                              -5.00
Goals_Conceded                      0.00
Missed_Passes                     -17.00
Missed_Penalty                      0.00
Offsides                           -4.00
Opponent_Team                Atlético-GO
Own_Goals                           0.00
Participated                        True
Penalty_Saves                       0.00
Points                            -10.00
Points_avg                         -8.30
PositionID                             1
Price                               0.71
Price_Variation                   -10.07
Red_Card                            0.00
Round           

In [11]:
# Removing entries with incorrect negative values
df_2017 = df_2017[df_2017['Assists'] >= 0]
df_2017 = df_2017[df_2017['Fouls_Conceded'] >= 0]
df_2017 = df_2017[df_2017['Fouls_Received'] >= 0]
df_2017 = df_2017[df_2017['Goals'] >= 0]
df_2017 = df_2017[df_2017['Missed_Passes'] >= 0]
df_2017 = df_2017[df_2017['Shot_Missed'] >= 0]
df_2017 = df_2017[df_2017['Shot_Saved'] >= 0]
df_2017 = df_2017[df_2017['Stolen_Balls'] >= 0]
df_2017 = df_2017[df_2017['Yellow_Card'] >= 0]
df_2017 = df_2017[df_2017['Offsides'] >= 0]

In [13]:
# Dropping values for which CLubID has NaN values.
df_2016 = df_2016.dropna(subset=['ClubID'])

Getting all the average personal scouts.

In [14]:
### WARNING - LENGTHY PROCESS ###
# For each target, get the averages; create new columns for each new feature average.

for target in targets:
    print(target)
    df_2014['avg_'+ target] = df_2014.apply(get_avg, args=(target, df_2014), axis=1)
    print('2014 done')
    df_2015['avg_'+ target] = df_2015.apply(get_avg, args=(target, df_2015), axis=1)
    print('2015 done')
    df_2016['avg_'+ target] = df_2016.apply(get_avg, args=(target, df_2016), axis=1)
    print('2016 done')
    df_2017['avg_'+ target] = df_2017.apply(get_avg, args=(target, df_2017), axis=1)
    print('2017 done')
    print('-------------------')
print('Completed!')

Assists
2014 done
2015 done
2016 done
2017 done
-------------------
Difficult_Saves
2014 done
2015 done
2016 done
2017 done
-------------------
Fouls_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Fouls_Received
2014 done
2015 done
2016 done
2017 done
-------------------
Game_Without_Being_Scored
2014 done
2015 done
2016 done
2017 done
-------------------
Goals
2014 done
2015 done
2016 done
2017 done
-------------------
Goals_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Missed_Passes
2014 done
2015 done
2016 done
2017 done
-------------------
Missed_Penalty
2014 done
2015 done
2016 done
2017 done
-------------------
Offsides
2014 done
2015 done
2016 done
2017 done
-------------------
Own_Goals
2014 done
2015 done
2016 done
2017 done
-------------------
Penalty_Saves
2014 done
2015 done
2016 done
2017 done
-------------------
Points
2014 done
2015 done
2016 done
2017 done
-------------------
Price
2014 done
2015 done
2016 done
2017 done
----

In [15]:
# Checkpoint spreadsheets.
df_2014.to_csv('Checkpoints/scouts_avg_2014.csv', index=False)
df_2015.to_csv('Checkpoints/scouts_avg_2015.csv', index=False)
df_2016.to_csv('Checkpoints/scouts_avg_2016.csv', index=False)
df_2017.to_csv('Checkpoints/scouts_avg_2017.csv', index=False)

In [16]:
clubs_2014 = df_2014['ClubID'].unique()
clubs_2015 = df_2015['ClubID'].unique()
clubs_2016 = df_2016['ClubID'].unique()
clubs_2017 = df_2017['ClubID'].unique()

In [17]:
rounds_2014 = df_2014['Round'].unique()
rounds_2015 = df_2015['Round'].unique()
rounds_2016 = df_2016['Round'].unique()
rounds_2017 = df_2017['Round'].unique()

In [18]:
# Get the sums for each of the seasons, preparing for the calculation of the team round average.
sum_2014 = df_2014.groupby(['Round', 'ClubID', 'PositionID'], as_index=False).sum()
sum_2015 = df_2015.groupby(['Round', 'ClubID', 'PositionID'], as_index=False).sum()
sum_2016 = df_2016.groupby(['Round', 'ClubID', 'PositionID'], as_index=False).sum()
sum_2017 = df_2017.groupby(['Round', 'ClubID', 'PositionID'], as_index=False).sum()

In [19]:
'''
Get the team average for each round for each team and position for the given season.
Creates a new column for each team-target-position.
'''

def get_team_round_avg(df, clubs, rounds, target, df_sums, year, position):
    for r in rounds:
        for club in clubs:
            check = (df_sums.Round == r) & (df_sums.ClubID == club) & (df_sums.PositionID == position)
            tot = df_sums.loc[check, target]
            if(len(tot) != 0):
                count = df_sums.loc[check, 'Season'].values[0] / year
                avg = tot.values[0]/count
            df.loc[((df['ClubID'] == club) & (df['Round'] == r)), 'team_avg_{}_{}'.format(position, target)] = avg

In [20]:
### WARNING - LENGTHY PROCESS ###
# For each position, for each target, get the team round average for that round.

for position in range(1,7):
    print('Position: %i' % (position))
    for target in targets:
        print(target)
        start_time = time.time()
        get_team_round_avg(df_2014, clubs_2014, rounds_2014, target, sum_2014, 2014, position)
        print('2014 done')
        get_team_round_avg(df_2015, clubs_2015, rounds_2015, target, sum_2015, 2015, position)
        print('2015 done')
        get_team_round_avg(df_2016, clubs_2016, rounds_2016, target, sum_2016, 2016, position)
        print('2016 done')
        get_team_round_avg(df_2017, clubs_2017, rounds_2017, target, sum_2017, 2017, position)
        print('2017 done')
        print('-------------------')
        elapsed_time = time.time() - start_time
        print('Time elapsed: %.2f seconds' % (elapsed_time))
        print('-------------------')
    print('Position Completed!')
    print('=======================')
print('Done!')

Position: 1
Assists
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.42 seconds
-------------------
Difficult_Saves
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.89 seconds
-------------------
Fouls_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.85 seconds
-------------------
Fouls_Received
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.08 seconds
-------------------
Game_Without_Being_Scored
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 58.34 seconds
-------------------
Goals
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.18 seconds
-------------------
Goals_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.15 seconds
-------------------
Missed_Passes
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.15 seconds
-------------------
Missed_Pe

2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.57 seconds
-------------------
Missed_Penalty
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.23 seconds
-------------------
Offsides
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.40 seconds
-------------------
Own_Goals
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.17 seconds
-------------------
Penalty_Saves
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.32 seconds
-------------------
Points
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 60.18 seconds
-------------------
Price
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.45 seconds
-------------------
Red_Card
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.06 seconds
-------------------
Shot_Missed
2014 done
2015 done
2016 done
2017 done
--------------

In [22]:
# Checkpoint spreadsheets.
df_2014.to_csv('Checkpoints/scouts_team_avg_2014.csv', index=False)
df_2015.to_csv('Checkpoints/scouts_team_avg_2015.csv', index=False)
df_2016.to_csv('Checkpoints/scouts_team_avg_2016.csv', index=False)
df_2017.to_csv('Checkpoints/scouts_team_avg_2017.csv', index=False)

In [23]:
df_2014.columns.values

array(['Assists', 'AthleteID', 'Athlete_Name', 'Away', 'ClubID',
       'Difficult_Saves', 'Fouls_Conceded', 'Fouls_Received',
       'Game_Without_Being_Scored', 'Goals', 'Goals_Conceded',
       'Missed_Passes', 'Missed_Penalty', 'Offsides', 'Opponent_Team',
       'Own_Goals', 'Participated', 'Penalty_Saves', 'Points',
       'Points_avg', 'PositionID', 'Price', 'Price_Variation', 'Red_Card',
       'Round', 'Season', 'Shot_Missed', 'Shot_Saved', 'Shot_on_Post',
       'Stolen_Balls', 'Team', 'Yellow_Card', 'opponentID', 'avg_Assists',
       'avg_Difficult_Saves', 'avg_Fouls_Conceded', 'avg_Fouls_Received',
       'avg_Game_Without_Being_Scored', 'avg_Goals', 'avg_Goals_Conceded',
       'avg_Missed_Passes', 'avg_Missed_Penalty', 'avg_Offsides',
       'avg_Own_Goals', 'avg_Penalty_Saves', 'avg_Points', 'avg_Price',
       'avg_Red_Card', 'avg_Shot_Missed', 'avg_Shot_Saved',
       'avg_Shot_on_Post', 'avg_Stolen_Balls', 'avg_Yellow_Card',
       'team_avg_1_Assists', 'team_avg_1_D

In [25]:
team_avg_column_targets = ['team_avg_1_Difficult_Saves',
       'team_avg_1_Points', 'team_avg_1_Assists',
       'team_avg_1_Fouls_Conceded', 'team_avg_1_Fouls_Received',
       'team_avg_1_Game_Without_Being_Scored',
       'team_avg_1_Goals_Conceded', 'team_avg_1_Missed_Passes',
       'team_avg_1_Penalty_Saves',
       'team_avg_1_Price', 'team_avg_1_Red_Card',
       'team_avg_1_Shot_Saved',
       'team_avg_1_Stolen_Balls', 'team_avg_1_Yellow_Card',
       'team_avg_2_Assists', 'team_avg_2_Fouls_Conceded',
       'team_avg_2_Fouls_Received', 'team_avg_2_Game_Without_Being_Scored',
       'team_avg_2_Goals',
       'team_avg_2_Missed_Passes', 'team_avg_2_Missed_Penalty',
       'team_avg_2_Offsides', 'team_avg_2_Own_Goals',
       'team_avg_2_Price','team_avg_2_Points',
       'team_avg_2_Red_Card', 'team_avg_2_Shot_Missed',
       'team_avg_2_Shot_Saved', 'team_avg_2_Shot_on_Post',
       'team_avg_2_Stolen_Balls', 'team_avg_2_Yellow_Card',
       'team_avg_3_Assists',
       'team_avg_3_Fouls_Conceded', 'team_avg_3_Fouls_Received',
       'team_avg_3_Game_Without_Being_Scored', 'team_avg_3_Goals',
       'team_avg_3_Missed_Passes',
       'team_avg_3_Missed_Penalty', 'team_avg_3_Offsides',
       'team_avg_3_Own_Goals',
       'team_avg_3_Points', 'team_avg_3_Price', 'team_avg_3_Red_Card',
       'team_avg_3_Shot_Missed', 'team_avg_3_Shot_Saved',
       'team_avg_3_Shot_on_Post', 'team_avg_3_Stolen_Balls',
       'team_avg_3_Yellow_Card', 'team_avg_4_Assists',
       'team_avg_4_Fouls_Conceded',
       'team_avg_4_Fouls_Received', 'team_avg_4_Game_Without_Being_Scored',
       'team_avg_4_Goals',
       'team_avg_4_Missed_Passes', 'team_avg_4_Missed_Penalty',
       'team_avg_4_Offsides', 'team_avg_4_Own_Goals',
       'team_avg_4_Points', 'team_avg_4_Price',
       'team_avg_4_Red_Card', 'team_avg_4_Shot_Missed',
       'team_avg_4_Shot_Saved', 'team_avg_4_Shot_on_Post',
       'team_avg_4_Stolen_Balls', 'team_avg_4_Yellow_Card',
       'team_avg_5_Assists',
       'team_avg_5_Fouls_Conceded', 'team_avg_5_Fouls_Received',
       'team_avg_5_Game_Without_Being_Scored', 'team_avg_5_Goals',
       'team_avg_5_Missed_Passes',
       'team_avg_5_Missed_Penalty', 'team_avg_5_Offsides',
       'team_avg_5_Own_Goals',
       'team_avg_5_Points', 'team_avg_5_Price', 'team_avg_5_Red_Card',
       'team_avg_5_Shot_Missed', 'team_avg_5_Shot_Saved',
       'team_avg_5_Shot_on_Post', 'team_avg_5_Stolen_Balls',
       'team_avg_5_Yellow_Card', 'team_avg_6_Points', 'team_avg_6_Price']

In [26]:
# Repeat process of getting averages but now for each team avg per position.
for target in team_avg_column_targets:
    start_time = time.time()
    print(target)
    get_avg_3(target, df_2014, clubs_2014, rounds_2014)
    print('2014 done')
    get_avg_3(target, df_2015, clubs_2015, rounds_2015)
    print('2015 done')
    get_avg_3(target, df_2016, clubs_2016, rounds_2016)
    print('2016 done')
    get_avg_3(target, df_2017, clubs_2017, rounds_2017)
    print('2017 done')
    print('-------------------')
    elapsed_time = time.time() - start_time
    print('Time elapsed: %.2f seconds' % (elapsed_time))
    print('-------------------')
print('Completed!')

team_avg_1_Difficult_Saves
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.89 seconds
-------------------
team_avg_1_Points
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.96 seconds
-------------------
team_avg_1_Assists
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.66 seconds
-------------------
team_avg_1_Fouls_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.05 seconds
-------------------
team_avg_1_Fouls_Received
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.09 seconds
-------------------
team_avg_1_Game_Without_Being_Scored
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.12 seconds
-------------------
team_avg_1_Goals_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.68 seconds
-------------------
team_avg_1_Missed_Passes
2014 done
2015 done
2016 done
2017 done


2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.11 seconds
-------------------
team_avg_4_Yellow_Card
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.20 seconds
-------------------
team_avg_5_Assists
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.99 seconds
-------------------
team_avg_5_Fouls_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 56.95 seconds
-------------------
team_avg_5_Fouls_Received
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.06 seconds
-------------------
team_avg_5_Game_Without_Being_Scored
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.24 seconds
-------------------
team_avg_5_Goals
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 57.32 seconds
-------------------
team_avg_5_Missed_Passes
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapse

In [27]:
# Checkpoint spreadsheets.
df_2014.to_csv('Checkpoints/team_avg_2014.csv', index=False)
df_2015.to_csv('Checkpoints/team_avg_2015.csv', index=False)
df_2016.to_csv('Checkpoints/team_avg_2016.csv', index=False)
df_2017.to_csv('Checkpoints/team_avg_2017.csv', index=False)

In [28]:
df_2014.columns.values

array(['Assists', 'AthleteID', 'Athlete_Name', 'Away', 'ClubID',
       'Difficult_Saves', 'Fouls_Conceded', 'Fouls_Received',
       'Game_Without_Being_Scored', 'Goals', 'Goals_Conceded',
       'Missed_Passes', 'Missed_Penalty', 'Offsides', 'Opponent_Team',
       'Own_Goals', 'Participated', 'Penalty_Saves', 'Points',
       'Points_avg', 'PositionID', 'Price', 'Price_Variation', 'Red_Card',
       'Round', 'Season', 'Shot_Missed', 'Shot_Saved', 'Shot_on_Post',
       'Stolen_Balls', 'Team', 'Yellow_Card', 'opponentID', 'avg_Assists',
       'avg_Difficult_Saves', 'avg_Fouls_Conceded', 'avg_Fouls_Received',
       'avg_Game_Without_Being_Scored', 'avg_Goals', 'avg_Goals_Conceded',
       'avg_Missed_Passes', 'avg_Missed_Penalty', 'avg_Offsides',
       'avg_Own_Goals', 'avg_Penalty_Saves', 'avg_Points', 'avg_Price',
       'avg_Red_Card', 'avg_Shot_Missed', 'avg_Shot_Saved',
       'avg_Shot_on_Post', 'avg_Stolen_Balls', 'avg_Yellow_Card',
       'team_avg_1_Assists', 'team_avg_1_D

In [29]:
team_avgs = ['avg_team_avg_1_Difficult_Saves',
       'avg_team_avg_1_Points', 'avg_team_avg_1_Assists',
       'avg_team_avg_1_Fouls_Conceded', 'avg_team_avg_1_Fouls_Received',
       'avg_team_avg_1_Game_Without_Being_Scored',
       'avg_team_avg_1_Goals_Conceded', 'avg_team_avg_1_Missed_Passes',
       'avg_team_avg_1_Penalty_Saves', 'avg_team_avg_1_Price',
       'avg_team_avg_1_Red_Card', 'avg_team_avg_1_Shot_Saved',
       'avg_team_avg_1_Stolen_Balls', 'avg_team_avg_1_Yellow_Card',
       'avg_team_avg_2_Assists', 'avg_team_avg_2_Fouls_Conceded',
       'avg_team_avg_2_Fouls_Received',
       'avg_team_avg_2_Game_Without_Being_Scored', 'avg_team_avg_2_Goals',
       'avg_team_avg_2_Missed_Passes', 'avg_team_avg_2_Missed_Penalty',
       'avg_team_avg_2_Offsides', 'avg_team_avg_2_Own_Goals',
       'avg_team_avg_2_Price', 'avg_team_avg_2_Points',
       'avg_team_avg_2_Red_Card', 'avg_team_avg_2_Shot_Missed',
       'avg_team_avg_2_Shot_Saved', 'avg_team_avg_2_Shot_on_Post',
       'avg_team_avg_2_Stolen_Balls', 'avg_team_avg_2_Yellow_Card',
       'avg_team_avg_3_Assists', 'avg_team_avg_3_Fouls_Conceded',
       'avg_team_avg_3_Fouls_Received',
       'avg_team_avg_3_Game_Without_Being_Scored', 'avg_team_avg_3_Goals',
       'avg_team_avg_3_Missed_Passes', 'avg_team_avg_3_Missed_Penalty',
       'avg_team_avg_3_Offsides', 'avg_team_avg_3_Own_Goals',
       'avg_team_avg_3_Points', 'avg_team_avg_3_Price',
       'avg_team_avg_3_Red_Card', 'avg_team_avg_3_Shot_Missed',
       'avg_team_avg_3_Shot_Saved', 'avg_team_avg_3_Shot_on_Post',
       'avg_team_avg_3_Stolen_Balls', 'avg_team_avg_3_Yellow_Card',
       'avg_team_avg_4_Assists', 'avg_team_avg_4_Fouls_Conceded',
       'avg_team_avg_4_Fouls_Received',
       'avg_team_avg_4_Game_Without_Being_Scored', 'avg_team_avg_4_Goals',
       'avg_team_avg_4_Missed_Passes', 'avg_team_avg_4_Missed_Penalty',
       'avg_team_avg_4_Offsides', 'avg_team_avg_4_Own_Goals',
       'avg_team_avg_4_Points', 'avg_team_avg_4_Price',
       'avg_team_avg_4_Red_Card', 'avg_team_avg_4_Shot_Missed',
       'avg_team_avg_4_Shot_Saved', 'avg_team_avg_4_Shot_on_Post',
       'avg_team_avg_4_Stolen_Balls', 'avg_team_avg_4_Yellow_Card',
       'avg_team_avg_5_Assists', 'avg_team_avg_5_Fouls_Conceded',
       'avg_team_avg_5_Fouls_Received',
       'avg_team_avg_5_Game_Without_Being_Scored', 'avg_team_avg_5_Goals',
       'avg_team_avg_5_Missed_Passes', 'avg_team_avg_5_Missed_Penalty',
       'avg_team_avg_5_Offsides', 'avg_team_avg_5_Own_Goals',
       'avg_team_avg_5_Points', 'avg_team_avg_5_Price',
       'avg_team_avg_5_Red_Card', 'avg_team_avg_5_Shot_Missed',
       'avg_team_avg_5_Shot_Saved', 'avg_team_avg_5_Shot_on_Post',
       'avg_team_avg_5_Stolen_Balls', 'avg_team_avg_5_Yellow_Card',
       'avg_team_avg_6_Points', 'avg_team_avg_6_Price']

In [30]:
'''
From the opponentID and ClubID cross reference and set the appropriate team average statistics.
'''
def set_opp_avg(row, target, df):
    opp_team_id = row['opponentID']
    r = row['Round']
    if(opp_team_id != 'NaN'):
        res = df.loc[((df['ClubID'] == opp_team_id) & (df['Round'] == r)), target].values
        if(len(res) > 0):
            return res[0]
    else:
        return 'NaN'

In [31]:
for target in team_avgs:
    start_time = time.time()
    
    print(target)
    df_2014['opp_'+ target] = df_2014.apply(set_opp_avg, args=(target, df_2014), axis=1)
    print('2014 done')
    df_2015['opp_'+ target] = df_2015.apply(set_opp_avg, args=(target, df_2015), axis=1)
    print('2015 done')
    df_2016['opp_'+ target] = df_2016.apply(set_opp_avg, args=(target, df_2016), axis=1)
    print('2016 done')
    df_2017['opp_'+ target] = df_2017.apply(set_opp_avg, args=(target, df_2017), axis=1)
    print('2017 done')
    
    print('-------------------')
    elapsed_time = time.time() - start_time
    print('Time elapsed: %.2f seconds' % (elapsed_time))
    print('-------------------')
print('Completed!')

avg_team_avg_1_Difficult_Saves
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 40.05 seconds
-------------------
avg_team_avg_1_Points
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 34.93 seconds
-------------------
avg_team_avg_1_Assists
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.69 seconds
-------------------
avg_team_avg_1_Fouls_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 36.09 seconds
-------------------
avg_team_avg_1_Fouls_Received
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.31 seconds
-------------------
avg_team_avg_1_Game_Without_Being_Scored
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.17 seconds
-------------------
avg_team_avg_1_Goals_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.04 seconds
-------------------
avg_team_avg_1_Missed_Passes
2014 don

2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.00 seconds
-------------------
avg_team_avg_4_Shot_on_Post
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.04 seconds
-------------------
avg_team_avg_4_Stolen_Balls
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.29 seconds
-------------------
avg_team_avg_4_Yellow_Card
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.39 seconds
-------------------
avg_team_avg_5_Assists
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.13 seconds
-------------------
avg_team_avg_5_Fouls_Conceded
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.17 seconds
-------------------
avg_team_avg_5_Fouls_Received
2014 done
2015 done
2016 done
2017 done
-------------------
Time elapsed: 35.35 seconds
-------------------
avg_team_avg_5_Game_Without_Being_Scored
2014 done
2015 done
2016 done
2017 do

In [32]:
# Checkpoint spreadsheets.
df_2014.to_csv('Checkpoints/opp_team_avg_2014.csv', index=False)
df_2015.to_csv('Checkpoints/opp_team_avg_2015.csv', index=False)
df_2016.to_csv('Checkpoints/opp_team_avg_2016.csv', index=False)
df_2017.to_csv('Checkpoints/opp_team_avg_2017.csv', index=False)

In [33]:
df_2014.columns.values

array(['Assists', 'AthleteID', 'Athlete_Name', 'Away', 'ClubID',
       'Difficult_Saves', 'Fouls_Conceded', 'Fouls_Received',
       'Game_Without_Being_Scored', 'Goals', 'Goals_Conceded',
       'Missed_Passes', 'Missed_Penalty', 'Offsides', 'Opponent_Team',
       'Own_Goals', 'Participated', 'Penalty_Saves', 'Points',
       'Points_avg', 'PositionID', 'Price', 'Price_Variation', 'Red_Card',
       'Round', 'Season', 'Shot_Missed', 'Shot_Saved', 'Shot_on_Post',
       'Stolen_Balls', 'Team', 'Yellow_Card', 'opponentID', 'avg_Assists',
       'avg_Difficult_Saves', 'avg_Fouls_Conceded', 'avg_Fouls_Received',
       'avg_Game_Without_Being_Scored', 'avg_Goals', 'avg_Goals_Conceded',
       'avg_Missed_Passes', 'avg_Missed_Penalty', 'avg_Offsides',
       'avg_Own_Goals', 'avg_Penalty_Saves', 'avg_Points', 'avg_Price',
       'avg_Red_Card', 'avg_Shot_Missed', 'avg_Shot_Saved',
       'avg_Shot_on_Post', 'avg_Stolen_Balls', 'avg_Yellow_Card',
       'team_avg_1_Assists', 'team_avg_1_D

In [34]:
# List of relevant parameters that are kept on the final dataframe.
final_params = ['AthleteID', 'Athlete_Name', 'Away', 'ClubID',
       'Opponent_Team', 'Participated', 'Points', 'PositionID', 'Price', 'Price_Variation',
       'Round', 'Season', 'Team', 'opponentID', 'avg_Assists',
       'avg_Difficult_Saves', 'avg_Fouls_Conceded', 'avg_Fouls_Received',
       'avg_Game_Without_Being_Scored', 'avg_Goals', 'avg_Goals_Conceded',
       'avg_Missed_Passes', 'avg_Missed_Penalty', 'avg_Offsides',
       'avg_Own_Goals', 'avg_Penalty_Saves', 'avg_Points', 'avg_Price',
       'avg_Red_Card', 'avg_Shot_Missed', 'avg_Shot_Saved',
       'avg_Shot_on_Post', 'avg_Stolen_Balls', 'avg_Yellow_Card',
       'avg_team_avg_1_Difficult_Saves',
       'avg_team_avg_1_Points', 'avg_team_avg_1_Assists',
       'avg_team_avg_1_Fouls_Conceded', 'avg_team_avg_1_Fouls_Received',
       'avg_team_avg_1_Game_Without_Being_Scored',
       'avg_team_avg_1_Goals_Conceded', 'avg_team_avg_1_Missed_Passes',
       'avg_team_avg_1_Penalty_Saves', 'avg_team_avg_1_Price',
       'avg_team_avg_1_Red_Card', 'avg_team_avg_1_Shot_Saved',
       'avg_team_avg_1_Stolen_Balls', 'avg_team_avg_1_Yellow_Card',
       'avg_team_avg_2_Assists', 'avg_team_avg_2_Fouls_Conceded',
       'avg_team_avg_2_Fouls_Received',
       'avg_team_avg_2_Game_Without_Being_Scored', 'avg_team_avg_2_Goals',
       'avg_team_avg_2_Missed_Passes', 'avg_team_avg_2_Missed_Penalty',
       'avg_team_avg_2_Offsides', 'avg_team_avg_2_Own_Goals',
       'avg_team_avg_2_Price', 'avg_team_avg_2_Points',
       'avg_team_avg_2_Red_Card', 'avg_team_avg_2_Shot_Missed',
       'avg_team_avg_2_Shot_Saved', 'avg_team_avg_2_Shot_on_Post',
       'avg_team_avg_2_Stolen_Balls', 'avg_team_avg_2_Yellow_Card',
       'avg_team_avg_3_Assists', 'avg_team_avg_3_Fouls_Conceded',
       'avg_team_avg_3_Fouls_Received',
       'avg_team_avg_3_Game_Without_Being_Scored', 'avg_team_avg_3_Goals',
       'avg_team_avg_3_Missed_Passes', 'avg_team_avg_3_Missed_Penalty',
       'avg_team_avg_3_Offsides', 'avg_team_avg_3_Own_Goals',
       'avg_team_avg_3_Points', 'avg_team_avg_3_Price',
       'avg_team_avg_3_Red_Card', 'avg_team_avg_3_Shot_Missed',
       'avg_team_avg_3_Shot_Saved', 'avg_team_avg_3_Shot_on_Post',
       'avg_team_avg_3_Stolen_Balls', 'avg_team_avg_3_Yellow_Card',
       'avg_team_avg_4_Assists', 'avg_team_avg_4_Fouls_Conceded',
       'avg_team_avg_4_Fouls_Received',
       'avg_team_avg_4_Game_Without_Being_Scored', 'avg_team_avg_4_Goals',
       'avg_team_avg_4_Missed_Passes', 'avg_team_avg_4_Missed_Penalty',
       'avg_team_avg_4_Offsides', 'avg_team_avg_4_Own_Goals',
       'avg_team_avg_4_Points', 'avg_team_avg_4_Price',
       'avg_team_avg_4_Red_Card', 'avg_team_avg_4_Shot_Missed',
       'avg_team_avg_4_Shot_Saved', 'avg_team_avg_4_Shot_on_Post',
       'avg_team_avg_4_Stolen_Balls', 'avg_team_avg_4_Yellow_Card',
       'avg_team_avg_5_Assists', 'avg_team_avg_5_Fouls_Conceded',
       'avg_team_avg_5_Fouls_Received',
       'avg_team_avg_5_Game_Without_Being_Scored', 'avg_team_avg_5_Goals',
       'avg_team_avg_5_Missed_Passes', 'avg_team_avg_5_Missed_Penalty',
       'avg_team_avg_5_Offsides', 'avg_team_avg_5_Own_Goals',
       'avg_team_avg_5_Points', 'avg_team_avg_5_Price',
       'avg_team_avg_5_Red_Card', 'avg_team_avg_5_Shot_Missed',
       'avg_team_avg_5_Shot_Saved', 'avg_team_avg_5_Shot_on_Post',
       'avg_team_avg_5_Stolen_Balls', 'avg_team_avg_5_Yellow_Card',
       'avg_team_avg_6_Points', 'avg_team_avg_6_Price',
       'opp_avg_team_avg_1_Difficult_Saves', 'opp_avg_team_avg_1_Points',
       'opp_avg_team_avg_1_Assists', 'opp_avg_team_avg_1_Fouls_Conceded',
       'opp_avg_team_avg_1_Fouls_Received',
       'opp_avg_team_avg_1_Game_Without_Being_Scored',
       'opp_avg_team_avg_1_Goals_Conceded',
       'opp_avg_team_avg_1_Missed_Passes',
       'opp_avg_team_avg_1_Penalty_Saves', 'opp_avg_team_avg_1_Price',
       'opp_avg_team_avg_1_Red_Card', 'opp_avg_team_avg_1_Shot_Saved',
       'opp_avg_team_avg_1_Stolen_Balls',
       'opp_avg_team_avg_1_Yellow_Card', 'opp_avg_team_avg_2_Assists',
       'opp_avg_team_avg_2_Fouls_Conceded',
       'opp_avg_team_avg_2_Fouls_Received',
       'opp_avg_team_avg_2_Game_Without_Being_Scored',
       'opp_avg_team_avg_2_Goals', 'opp_avg_team_avg_2_Missed_Passes',
       'opp_avg_team_avg_2_Missed_Penalty', 'opp_avg_team_avg_2_Offsides',
       'opp_avg_team_avg_2_Own_Goals', 'opp_avg_team_avg_2_Price',
       'opp_avg_team_avg_2_Points', 'opp_avg_team_avg_2_Red_Card',
       'opp_avg_team_avg_2_Shot_Missed', 'opp_avg_team_avg_2_Shot_Saved',
       'opp_avg_team_avg_2_Shot_on_Post',
       'opp_avg_team_avg_2_Stolen_Balls',
       'opp_avg_team_avg_2_Yellow_Card', 'opp_avg_team_avg_3_Assists',
       'opp_avg_team_avg_3_Fouls_Conceded',
       'opp_avg_team_avg_3_Fouls_Received',
       'opp_avg_team_avg_3_Game_Without_Being_Scored',
       'opp_avg_team_avg_3_Goals', 'opp_avg_team_avg_3_Missed_Passes',
       'opp_avg_team_avg_3_Missed_Penalty', 'opp_avg_team_avg_3_Offsides',
       'opp_avg_team_avg_3_Own_Goals', 'opp_avg_team_avg_3_Points',
       'opp_avg_team_avg_3_Price', 'opp_avg_team_avg_3_Red_Card',
       'opp_avg_team_avg_3_Shot_Missed', 'opp_avg_team_avg_3_Shot_Saved',
       'opp_avg_team_avg_3_Shot_on_Post',
       'opp_avg_team_avg_3_Stolen_Balls',
       'opp_avg_team_avg_3_Yellow_Card', 'opp_avg_team_avg_4_Assists',
       'opp_avg_team_avg_4_Fouls_Conceded',
       'opp_avg_team_avg_4_Fouls_Received',
       'opp_avg_team_avg_4_Game_Without_Being_Scored',
       'opp_avg_team_avg_4_Goals', 'opp_avg_team_avg_4_Missed_Passes',
       'opp_avg_team_avg_4_Missed_Penalty', 'opp_avg_team_avg_4_Offsides',
       'opp_avg_team_avg_4_Own_Goals', 'opp_avg_team_avg_4_Points',
       'opp_avg_team_avg_4_Price', 'opp_avg_team_avg_4_Red_Card',
       'opp_avg_team_avg_4_Shot_Missed', 'opp_avg_team_avg_4_Shot_Saved',
       'opp_avg_team_avg_4_Shot_on_Post',
       'opp_avg_team_avg_4_Stolen_Balls',
       'opp_avg_team_avg_4_Yellow_Card', 'opp_avg_team_avg_5_Assists',
       'opp_avg_team_avg_5_Fouls_Conceded',
       'opp_avg_team_avg_5_Fouls_Received',
       'opp_avg_team_avg_5_Game_Without_Being_Scored',
       'opp_avg_team_avg_5_Goals', 'opp_avg_team_avg_5_Missed_Passes',
       'opp_avg_team_avg_5_Missed_Penalty', 'opp_avg_team_avg_5_Offsides',
       'opp_avg_team_avg_5_Own_Goals', 'opp_avg_team_avg_5_Points',
       'opp_avg_team_avg_5_Price', 'opp_avg_team_avg_5_Red_Card',
       'opp_avg_team_avg_5_Shot_Missed', 'opp_avg_team_avg_5_Shot_Saved',
       'opp_avg_team_avg_5_Shot_on_Post',
       'opp_avg_team_avg_5_Stolen_Balls',
       'opp_avg_team_avg_5_Yellow_Card', 'opp_avg_team_avg_6_Points',
       'opp_avg_team_avg_6_Price']

In [35]:
df_2014 = df_2014[final_params]
df_2015 = df_2015[final_params]
df_2016 = df_2016[final_params]
df_2017 = df_2017[final_params]

In [36]:
df_all = df_2014.copy()
df_all = df_all.append(df_2015)
df_all = df_all.append(df_2016)

In [37]:
# Dropping entries where avg_points in NaN, including the first round for all seasons.
df_all = df_all.dropna(how='all',subset=['avg_Points'])
df_2017 = df_2017.dropna(how='all',subset=['avg_Points'])

In [38]:
# Partitioning by position.
df_1_all = df_all[df_all['PositionID'] == 1]
df_2_all = df_all[df_all['PositionID'] == 2]
df_3_all = df_all[df_all['PositionID'] == 3]
df_4_all = df_all[df_all['PositionID'] == 4]
df_5_all = df_all[df_all['PositionID'] == 5]
df_6_all = df_all[df_all['PositionID'] == 6]

In [39]:
df_1_2017 = df_2017[df_2017['PositionID'] == 1]
df_2_2017 = df_2017[df_2017['PositionID'] == 2]
df_3_2017 = df_2017[df_2017['PositionID'] == 3]
df_4_2017 = df_2017[df_2017['PositionID'] == 4]
df_5_2017 = df_2017[df_2017['PositionID'] == 5]
df_6_2017 = df_2017[df_2017['PositionID'] == 6]

### Removing NaN containing rows/columns

In [40]:
df_1_all = df_1_all.dropna(how='all', axis=1)
df_2_all = df_2_all.dropna(how='all', axis=1) 
df_3_all = df_3_all.dropna(how='all', axis=1)
df_4_all = df_4_all.dropna(how='all', axis=1)
df_5_all = df_5_all.dropna(how='all', axis=1)
df_6_all = df_6_all.dropna(how='all', axis=1)

df_1_2017 = df_1_2017.dropna(how='all', axis=1)
df_2_2017 = df_2_2017.dropna(how='all', axis=1) 
df_3_2017 = df_3_2017.dropna(how='all', axis=1) 
df_4_2017 = df_4_2017.dropna(how='all', axis=1) 
df_5_2017 = df_5_2017.dropna(how='all', axis=1) 
df_6_2017 = df_6_2017.dropna(how='all', axis=1)

In [41]:
df_1_all = df_1_all.dropna(how='any', axis=0)
df_2_all = df_2_all.dropna(how='any', axis=0) 
df_3_all = df_3_all.dropna(how='any', axis=0)
df_4_all = df_4_all.dropna(how='any', axis=0)
df_5_all = df_5_all.dropna(how='any', axis=0)
df_6_all = df_6_all.dropna(how='any', axis=0)

df_1_2017 = df_1_2017.dropna(how='any', axis=0)
df_2_2017 = df_2_2017.dropna(how='any', axis=0) 
df_3_2017 = df_3_2017.dropna(how='any', axis=0) 
df_4_2017 = df_4_2017.dropna(how='any', axis=0) 
df_5_2017 = df_5_2017.dropna(how='any', axis=0) 
df_6_2017 = df_6_2017.dropna(how='any', axis=0)

In [42]:
print(df_1_all.shape)
print(df_2_all.shape)
print(df_3_all.shape)
print(df_4_all.shape)
print(df_5_all.shape)
print(df_6_all.shape)

(2015, 202)
(4040, 202)
(4126, 202)
(9736, 202)
(6516, 202)
(2022, 202)


In [43]:
print(df_1_2017.shape)
print(df_2_2017.shape)
print(df_3_2017.shape)
print(df_4_2017.shape)
print(df_5_2017.shape)
print(df_6_2017.shape)

(663, 202)
(1370, 202)
(1373, 202)
(3277, 202)
(2059, 202)
(671, 202)


In [44]:
df_1_all.to_csv('Cleaned Data/df_1_all_reg.csv', index=False)
df_2_all.to_csv('Cleaned Data/df_2_all_reg.csv', index=False)
df_3_all.to_csv('Cleaned Data/df_3_all_reg.csv', index=False)
df_4_all.to_csv('Cleaned Data/df_4_all_reg.csv', index=False)
df_5_all.to_csv('Cleaned Data/df_5_all_reg.csv', index=False)
df_6_all.to_csv('Cleaned Data/df_6_all_reg.csv', index=False)

In [45]:
df_1_2017.to_csv('Cleaned Data/df_1_2017_reg.csv', index=False)
df_2_2017.to_csv('Cleaned Data/df_2_2017_reg.csv', index=False)
df_3_2017.to_csv('Cleaned Data/df_3_2017_reg.csv', index=False)
df_4_2017.to_csv('Cleaned Data/df_4_2017_reg.csv', index=False)
df_5_2017.to_csv('Cleaned Data/df_5_2017_reg.csv', index=False)
df_6_2017.to_csv('Cleaned Data/df_6_2017_reg.csv', index=False)